In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
from qutip import *
from estimation import *
from data_processing import *
from scipy.io import savemat
from targets import *
import exp_params

USING W-FUNCTION POINTS


### Check yourself: print function, dimension, and number of displacements

In [2]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

W
D=6
nD=35


### State list

In [3]:
state_list = exp_params.fock_state_list
print(state_list)

['fock0' 'fock01' 'fock0i1' 'fock02' 'fock0i2' 'fock1' 'fock03' 'fock0i3'
 'fock12' 'fock1i2' 'fock04' 'fock0i4' 'fock13' 'fock1i3' 'fock2' 'fock05'
 'fock0i5' 'fock14' 'fock1i4' 'fock23' 'fock2i3' 'fock15' 'fock1i5'
 'fock24' 'fock2i4' 'fock3' 'fock25' 'fock2i5' 'fock34' 'fock3i4' 'fock35'
 'fock3i5' 'fock4' 'fock45' 'fock4i5' 'fock5']


### Experimental data directory

In [4]:
exp_data_directory = r"data\fidelity_v_dimension\W\D" + str(exp_params.D)
all_files = np.array(os.listdir(exp_data_directory))

### Directory for target states generated from grape (we don't use ideal when we're processing experimental data)

In [5]:
# GRAPE generated target states
target_states = r"target_states" # path to directory of target states generated from grape

### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [6]:

map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_W.npz"
)


W = map_variables["W"]

beta = map_variables["beta"]

### Iterate through the states. For each state: 1) collect the observable outcomes, 2) perform linear inversion with the map to get $\rho_{LS}$, 3) perform both MLE and Bayesian, and 4) record in numpy arrays the fidelities, standard deviations, and $\rho_{BME}$ for each. This saves both the normal Wigner and corrected Wigner information.

In [7]:
F_MLE_corr = np.zeros(len(state_list), dtype=float)
F_MLE_norm = np.zeros(len(state_list), dtype=float)


F_Bayes_BME_corr = np.zeros(len(state_list), dtype=float)
F_Bayes_BME_norm = np.zeros(len(state_list), dtype=float)
final_rho_BME_corr = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)
final_rho_BME_norm = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)
final_rho_MLE_corr = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)
final_rho_MLE_norm = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)

In [8]:
for j, state_name in enumerate(state_list):  # State List
    data_corr = np.zeros([exp_params.nD])  # Initialize data vector
    data_norm = np.zeros([exp_params.nD])  # Initialize data vector

    for point in range(1, exp_params.nD + 1):  # Point1 to Point35

        ending = (
            "_grape_"
            + str(state_name)
            + "_point"
            + str(point)
            + ".h5"
        )
        matching = [
            filename for filename in all_files if filename.endswith(str(ending))
        ]

        filepath = exp_data_directory + "/" + matching[0]
        file = h5py.File(filepath, "r")
        # selected_data, selected_data_m = select2_flat(filepath, postSel)
        signal, signal_m = post_selection_W(filepath)
        # signal_m = np.average(selected_data_m)

        if state_name in exp_params.pes_after_grape[:, 0]:
            pge = exp_params.pes_after_grape[
                (exp_params.pes_after_grape[:, 0] == state_name), 1
            ].astype(float)

        else:
            pge = exp_params.pe_after_grape_avg

        aux = (signal - pge) / (1 - 2 * pge)
        aux_m = (signal_m - pge) / (1 - 2 * pge)

        data_corr[point - 1] = aux - aux_m
        data_norm[point - 1] = 2 * aux - 1

    rho_est_corr, qRho_est_corr = get_LS_and_MLE_rho_est(
        data_corr, W, beta, exp_params.D, exp_params.nD
    )
    rho_est_norm, qRho_est_norm = get_LS_and_MLE_rho_est(
        data_norm, W, beta, exp_params.D, exp_params.nD
    )

    rho_tar_qc = Y_target(state_name, target_states, qdim=3, cdim=30)

    rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])
    rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()

    Fmean_bayes_corr, Fstd_bayes_corr, rho_BME_corr = bayesian_rho_est(
        numSamp=2**10,
        N=exp_params.N_exp,
        rho_tar=rho_tar_c,
        rhoLS=rho_est_corr.full(),
    )
    Fmean_bayes_norm, Fstd_bayes_norm, rho_BME_norm = bayesian_rho_est(
        numSamp=2**10,
        N=exp_params.N_exp,
        rho_tar=rho_tar_c,
        rhoLS=rho_est_norm.full(),
    )
    F_Bayes_BME_corr[j] = fidelity(rho_tar_c, Qobj(rho_BME_corr)) ** 2
    F_Bayes_BME_norm[j] = fidelity(rho_tar_c, Qobj(rho_BME_norm)) ** 2

    F_MLE_corr[j] = fidelity(rho_tar_c, qRho_est_corr) ** 2
    F_MLE_norm[j] = fidelity(rho_tar_c, qRho_est_norm) ** 2

    final_rho_BME_corr[j, :, :] = rho_BME_corr
    final_rho_BME_norm[j, :, :] = rho_BME_norm

    final_rho_MLE_corr[j, :, :] = qRho_est_corr
    final_rho_MLE_norm[j, :, :] = qRho_est_norm

    print(f"state {j}: {state_name}, corr: {F_Bayes_BME_corr[j]}")
    print(f"state {j}: {state_name}, norm: {F_Bayes_BME_norm[j]}\n")

C:\Users\clara\AppData\Local\Temp\ipykernel_7336\1733429237.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data_corr[point - 1] = aux - aux_m
C:\Users\clara\AppData\Local\Temp\ipykernel_7336\1733429237.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data_norm[point - 1] = 2 * aux - 1


state 0: fock0, corr: 0.9029379896178136
state 0: fock0, norm: 0.8368058097286523



### Save results in an npz

In [ ]:
np.savez(
    f"results_dimensions\exp\W\Bayes_F_D={exp_params.D}_nD={exp_params.nD}_BME_MLE_rho_corr_norm_W.npz",
    F_Bayes_BME_corr=F_Bayes_BME_corr,
    F_Bayes_BME_norm=F_Bayes_BME_norm,
    F_MLE_corr=F_MLE_corr,
    F_MLE_norm=F_MLE_norm,
    final_rho_BME_corr=final_rho_BME_corr,
    final_rho_BME_norm=final_rho_BME_norm,
    final_rho_MLE_corr=final_rho_MLE_corr,
    final_rho_MLE_norm=final_rho_MLE_norm,
)